In [1]:
import pandas as pd

df = pd.read_csv(r"C:\Users\anime\OneDrive\Desktop\Data Science Projects\Youtube Comment Analysis\youtube_comment_analyser\data\processed\reddit_preprocessing.csv")
df

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1
...,...,...
36788,jesus,0
36789,kya bhai pure saal chutiya banaya modi aur jab...,1
36790,downvote karna tha par upvote hogaya,0
36791,haha nice,1


In [2]:
df =  df.dropna(subset=["clean_comment"])
df.shape

(36661, 2)

In [3]:
pip install mlflow boto3 awscli optuna xgboost imbalanced-learn

  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/150.0 MB 3.7 MB/s eta 0:00:40
   ---------------------------------------- 1.3/150.0 MB 2.9 MB/s eta 0:00:51
   ---------------------------------------- 1.6/150.0 MB 2.4 MB/s eta 0:01:02
    --------------------------------------- 2.1/150.0 MB 2.5 MB/s eta 0:01:00
    --------------------------------------- 2.4/150.0 MB 2.2 MB/s eta 0:01:06
    --------------------------------------- 2.6/150.0 MB 2.0 MB/s eta 0:01:15
    --------------------------------------- 2.6/150.0 MB 2.0 MB/s eta 0:01:15
    --------------------------------------- 3.1/150.0 MB 1.8 MB/s eta 0:01:21
    --------------------------------------- 3.4/150.0 MB 1.7 MB/s eta 0:01:25
    --------------------------------------- 3.4/150.0 MB 1.7 MB/s eta 0:01:25
   - -------------------------------------- 3.9/150.0 MB 1.6 MB/s eta 0:01:30
   - --

In [ ]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

c:\Users\anime\OneDrive\Desktop\Data Science Projects\Youtube Comment Analysis\comment_analysis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
mlflow.set_experiment("ML Algos with Revised HP Tuning")

<Experiment: artifact_location='s3://datascienceanimesh/516785590309530379', creation_time=1743657286448, experiment_id='516785590309530379', last_update_time=1743657286448, lifecycle_stage='active', name='ML Algos with Revised HP Tuning', tags={}>

In [5]:
import mlflow
import optuna
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: Train-test split before vectorization and resampling
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# Step 4: Vectorization using TF-IDF (fit only on training data)
ngram_range = (1, 3)  # Use trigram features
max_features = 1000  # Limit vocabulary size

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train_raw)  # Fit on training data
X_test_vec = vectorizer.transform(X_test_raw)  # Transform test data

# Step 5: Apply undersampling to balance the training set
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_Undersampling_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost using cross-validation
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(
        n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42
    )

    # Use cross-validation on resampled training data
    cv_scores = cross_val_score(model, X_train_resampled, y_train_resampled, cv=5, scoring='accuracy')
    return np.mean(cv_scores)  # Return mean accuracy of cross-validation


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'],
        random_state=42
    )

    # Train the best model on the resampled training set
    best_model.fit(X_train_resampled, y_train_resampled)

    # Log the best model using MLflow
    log_mlflow("XGBoost", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)


# Run the experiment for XGBoost
run_optuna_experiment()


C:\Users\anime\AppData\Local\Temp\ipykernel_18020\4136554005.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})
[I 2025-04-03 18:40:42,215] A new study created in memory with name: no-name-64d8af83-f3da-40ca-a273-3168a804f2da
[I 2025-04-03 18:43:03,445] Trial 0 finished with value: 0.5959383708275106 and parameters: {'n_estimators': 243, 'learning_rate': 0.004424780522563613, 'max_depth': 5}. Best is trial 0 with value: 0.5959383708275106.
[I 2025-04-03 18:47:51,686] Trial 1 finished with value: 0.5978580092231375 and parameters: {'n_estimators': 208, 'learning_rate': 0.0008520071167396027, 'max_depth': 8}. Best is trial 1 with value: 0.5978580092231375.
[I 2025-04-03 18:49:50,099] Trial 

🏃 View run XGBoost_Undersampling_TFIDF_Trigrams at: http://ec2-16-171-237-43.eu-north-1.compute.amazonaws.com:5000/#/experiments/516785590309530379/runs/8c4b712285bb442b868ab4c99d30c300
🧪 View experiment at: http://ec2-16-171-237-43.eu-north-1.compute.amazonaws.com:5000/#/experiments/516785590309530379
